# Sprint 1: Data analysis and supervised learning

## Opmerkingen dataset

* Complaint volume moet in verband worden gebracht met de grootte en het marktaandeel van het bedrijf.
Een bedrijf met een groot aantal klanten zal bijvoorbeeld meer klachten hebben dan één met een klein aantal klanten.

## Inlezen dataset

In [1]:
import pandas as pd
import csv

def read_all_complaints() :
    # Store all the data of the complaints in an array of maps. (amount of maps = amount of complaints)
    all_complaints = []
    with open('data/complaints-100.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print("All the information stored about a complaint:")
                for category in row:
                    print("-" + category)
                line_count += 1
            else: 
                complaint = {}
                complaint["product"] = row[1]
                complaint["sub-product"] = row[1]
                all_complaints.append(complaint)
                line_count += 1
        print(f'File processed of {line_count} lines.')
        return all_complaints
    csv_file.close()
    
complaints = read_all_complaints()

All the information stored about a complaint:
-Date received
-Product
-Sub-product
-Issue
-Sub-issue
-Consumer complaint narrative
-Company public response
-Company
-State
-ZIP code
-Tags
-Consumer consent provided?
-Submitted via
-Date sent to company
-Company response to consumer
-Timely response?
-Consumer disputed?
-Complaint ID
File processed of 32 lines.


## Bepalen van het aantal producten en sub-producten

In [2]:
def give_product_complaint_amount():
    products = {}
    for complaint in complaints:
        product=complaint["product"]
        if product in products:
            products[complaint["product"]] +=1
        else:
            products[complaint["product"]] = 0
    for product in products:
        amount = str(products[product])
    return pd.DataFrame(products.items(), columns=["Product", "Amount"]) 

In [3]:
def give_subproduct_complaint_amount():
    sub_products = {}
    for complaint in complaints:
        sub_product=complaint["sub-product"]
        if sub_product in sub_products:
            sub_products[complaint["sub-product"]] +=1
        else:
            sub_products[complaint["sub-product"]] = 0
    for sub_product in sub_products:
        amount = str(sub_products[sub_product])
    return pd.DataFrame(sub_products.items(), columns=["Sub-Product", "Amount"]) 

In [5]:
print(give_product_complaint_amount())
print(give_subproduct_complaint_amount())

                                             Product  Amount
0  Credit reporting, credit repair services, or o...      16
1                                    Debt collection       8
2          Payday loan, title loan, or personal loan       1
3                                           Mortgage       0
4                        Credit card or prepaid card       0
5                        Checking or savings account       0
                                         Sub-Product  Amount
0  Credit reporting, credit repair services, or o...      16
1                                    Debt collection       8
2          Payday loan, title loan, or personal loan       1
3                                           Mortgage       0
4                        Credit card or prepaid card       0
5                        Checking or savings account       0
